In [ ]:
# !pip install mecab-python3
# !pip install unidic-lite
# !pip install nltk
# !pip install scikit-learn
# !pip install seaborn

In [ ]:
# !pip install BeautifulSoup4
# !pip install neologdn
# !pip install stylecloud
# !pip install scrapy
# !pip install selenium

In [ ]:
# import xlwt
from bs4 import BeautifulSoup as bs
from urllib import request
import re
import scrapy

In [ ]:
https://www.cnblogs.com/yoyoketang/p/6557421.html

workbook = xlwt.Workbook()
worksheet = workbook.add_sheet('filmarks',cell_overwrite_ok = True)

worksheet.write(0,0,label='レビュアー')
worksheet.write(0,1,label='時間')
worksheet.write(0,2,label='内容')

workbook.save('filmarks.xls')

### ①クロージング

In [1]:
from selenium import webdriver
import time
browser = webdriver.Chrome()

browser.get('https://filmarks.com/movies/86613')
time.sleep(3)

#https://blog.csdn.net/zhangvalue/article/details/102921631
links = []

for link in browser.find_elements_by_xpath("//*[@href]"):
    if "page" in str(link.get_attribute('href')):
        links.append(link.get_attribute('href'))

In [2]:
times = links[-1].split("page=")[-1]
times

'303'

In [ ]:
browser.get('https://filmarks.com/movies/86613?page=298')
# time.sleep(3)
user_list = []
condition_list = []
for test in browser.find_elements_by_tag_name('img'):
    user_list.append(test.get_attribute('alt'))
    condition_list .append(test.get_attribute('loading'))

In [ ]:
users = user_list[3:13]
conditions = condition_list[3:13]

In [52]:
texts = []
scores = []
users = []
conditions = []

for i in range(1,int(times)+1):
    browser.get('https://filmarks.com/movies/86613?page='+str(i))
    element_text = browser.find_elements('css selector','.p-mark__review')
    element_score = browser.find_elements('css selector','.c-rating__score')
    text = [m.text for m in element_text]
    score = [l.text for l in element_score]
    user = [n.get_attribute('alt') for n in browser.find_elements_by_tag_name('img')]
    condition = [c.get_attribute('loading') for c in browser.find_elements_by_tag_name('img')]
    texts.append(text)
    scores.append(score[1:11])
    users.append(user[3:13])
    conditions.append(condition[3:13])  

In [53]:
print(len(sum(texts,[])))
print(len(sum(users,[])))
print(len(sum(scores,[])))
print(len(sum(conditions,[])))
print(len(texts))
print(len(users))
print(len(scores))
print(len(conditions))

3030
3030
3030
3030
303
303
303
303


In [75]:
import pandas as pd
# df_datas = pd.DataFrame([]texts,columns =['text'])
df_datas = pd.DataFrame([sum(users,[]),sum(texts,[]),sum(scores,[]),sum(conditions,[])],index =['users','texts','scores','conditions'])

In [76]:
df_datas.T.to_csv('luoxiaohei_datas.csv',encoding ='utf-8-sig')

In [77]:
df = df_datas.T
df.head(2)

,users,texts,scores,conditions
0,Amy,シャオヘイが本当かわいい\nどのキャラも魅力的\n\n全体的に絵が柔らかくて可愛い\nアクシ...,3.5,lazy
1,はる,絵が綺麗で、アクションも迫力あった！\n轟音上映で観れてよかった！\nそして何よりシャオヘイ...,4.7,lazy


### ②前処理

In [78]:
#emojiの除去
import neologdn
import re
def filter(desstr, restr=''):
    """
    filter emoji
    desstr: origin str
    restr: replace str
    """
    # filter emoji
    res = re.compile(u'[\U00010000-\U0010ffff]')
    res_emoji = res.sub(restr,desstr)
    res_linebreak = ''.join(res_emoji.replace('<br/>','').split())
    res_uppertolower = res_linebreak.lower()
    res_normalization = neologdn.normalize(res_uppertolower)
    res_number = re.sub('[①-⑨]', '',res_normalization)
    res_url = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+','',res_number)
        
    return res_url

In [79]:
text_ed = [filter(i) for i in df['texts']]

In [80]:
df['texts_ed'] = text_ed
df.head(2)

,users,texts,scores,conditions,texts_ed
0,Amy,シャオヘイが本当かわいい\nどのキャラも魅力的\n\n全体的に絵が柔らかくて可愛い\nアクシ...,3.5,lazy,シャオヘイが本当かわいいどのキャラも魅力的全体的に絵が柔らかくて可愛いアクションシーンはスピ...
1,はる,絵が綺麗で、アクションも迫力あった！\n轟音上映で観れてよかった！\nそして何よりシャオヘイ...,4.7,lazy,絵が綺麗で、アクションも迫力あった!轟音上映で観れてよかった!そして何よりシャオヘイが可愛く...


### 分かち書き

In [81]:
import MeCab

In [82]:
#名詞、動詞、形容詞の抽出
# 参考＜https://qiita.com/chamao/items/7edaba62b120a660657e＞
def wakati_by_mecab(text):
    tagger = MeCab.Tagger('')
    tagger.parse('') 
    node = tagger.parseToNode(text)
    word_list = []
    while node:
        pos = node.feature.split(",")[0]
#         if pos in ["形容詞"]:   # 対象とする品詞

        if pos in ["名詞", "動詞", "形容詞"]:   # 対象とする品詞
            word = node.surface
            word_list.append(word)
        node = node.next
    return " ".join(word_list)

In [84]:
texts_wakati = [wakati_by_mecab(i) for i in text_ed]

In [86]:
texts_wakati

'絵 アクション 迫力 あっ 轟音 上映 観れ よかっ シャオヘイ 可愛く ムゲン かっこ いい'

In [87]:
df['texts_wakati'] =texts_wakati

In [89]:
df.head(2)

,users,texts,scores,conditions,texts_ed,texts_wakati
0,Amy,シャオヘイが本当かわいい\nどのキャラも魅力的\n\n全体的に絵が柔らかくて可愛い\nアクシ...,3.5,lazy,シャオヘイが本当かわいいどのキャラも魅力的全体的に絵が柔らかくて可愛いアクションシーンはスピ...,シャオヘイ 本当 かわいい キャラ 魅力 全体 絵 柔らかく 可愛い アクション シーン ス...
1,はる,絵が綺麗で、アクションも迫力あった！\n轟音上映で観れてよかった！\nそして何よりシャオヘイ...,4.7,lazy,絵が綺麗で、アクションも迫力あった!轟音上映で観れてよかった!そして何よりシャオヘイが可愛く...,絵 アクション 迫力 あっ 轟音 上映 観れ よかっ シャオヘイ 可愛く ムゲン かっこ いい


### 極性辞書の読み込み

In [92]:
dict_= pd.read_table('極性辞書.txt',sep =':')

In [93]:
dict_

,優れる,すぐれる,動詞,1
0,良い,よい,形容詞,0.999995
1,喜ぶ,よろこぶ,動詞,0.999979
2,褒める,ほめる,動詞,0.999979
3,めでたい,めでたい,形容詞,0.999645
4,賢い,かしこい,形容詞,0.999486
...,...,...,...,...
55119,ない,ない,助動詞,-0.999997
55120,酷い,ひどい,形容詞,-0.999997
55121,病気,びょうき,名詞,-0.999998
55122,死ぬ,しぬ,動詞,-0.999999


In [ ]:
#改行記号などの除去
def linebreak(desstr):
    return ''.join(desstr.replace('<br/>','').split())

In [ ]:
texts_ed = [filter_emoji(i) for i in df['comments']]

In [ ]:
texts_ed = [linebreak(i) for i in texts_ed]

In [ ]:
texts_ed[1]

In [ ]:
#大文字を小文字に統一する
def uppertolower(desstr):
    return desstr.lower()

In [ ]:
texts_ed = [uppertolower(i) for i in texts_ed]

In [ ]:
texts_ed[0]

In [ ]:
#正規化
def normalization(desstr):
    return neologdn.normalize(desstr)

In [ ]:
import neologdn
texts_ed = [normalization(i) for i in texts_ed]

In [ ]:
texts_ed[0]

In [ ]:
#記号系数字の除去
def number(desstr):
    return re.sub('[①-⑨]', '', desstr)

In [ ]:
texts_ed = [number(i) for i in texts_ed]

In [ ]:
texts_ed[0]

In [ ]:
#urlの除去
def url(desstr):
    return re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+','',desstr)

In [ ]:
texts_ed = [url(i) for i in texts_ed]

In [ ]:
texts_ed[504]

### ご参考まで：

In [ ]:
elements = browser.find_elements('css selector','.p-mark__review') 
print(type(elements))
print(elements)

In [ ]:
print(element[3].text)

In [ ]:
browser.quit()

In [ ]:
elements_page = browser.find_elements('css selector','.c-pagination__prev') 
print(type(elements_page))
print(elements_page)

In [ ]:
print(elements_page[0].text)

#### selenium.common.exceptions.WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home
    対応方法：https://blog.csdn.net/qq_38486203/article/details/82852240
    ＜重启也不行，把chromedriver.exe放到python脚本的文件夹下面，再执行试试＞→OK

In [ ]:
url = 'https://filmarks.com/movies/86613/no_spoiler?page='
# url_num = re.search('(.*?)page=[0-9]',url)
html_data = []
for i in range(1,175):
#     url = get_page(url_basic)
    resp = request.urlopen(url+str(i))
    data = resp.read().decode('utf-8')
    html_data.append(data)

In [ ]:
html_data[3]

In [ ]:
#参考<https://www.cnblogs.com/cymwill/articles/7574479.html>
pmark_review_lists = []
pmark_user_lists = []
pmark_score_lists = []
for m in html_data:
    soup = bs(m,'html.parser')
    #コメント＜soup＞
    pmark_review_list = soup.find_all('div','p-mark__review')
    
    #ユーザー＜soup＞
    pmark_user_list = soup.find_all(lambda tag:tag.has_attr('alt') and tag.has_attr('loading'))
    
    #スコア＜正規＞
    pmark_score_total = soup.find_all('div',class_='c-media__content')
    pmark_score_total_list = []
    for i in pmark_score_total:
        pmark_score_total_list.append(re.findall('<div class="c-rating__score">(.*)</div></div></div>',str(i)))

    pmark_review_lists.append(pmark_review_list)
    pmark_user_lists.append(pmark_user_list)
    pmark_score_lists.append(pmark_score_total_list) 

In [ ]:
 pmark_score_lists

In [ ]:
#参考《https://blog.csdn.net/u012611644/article/details/82814377》
import re

comments = []
for item in sum(pmark_review_lists,[]):
    item_ed = re.findall('<div class="p-mark__review">(.*)</div>',str(item))
    comments.append(item_ed)

users =[]
for user in sum(pmark_user_lists,[]):
    user_ed = re.findall('<img alt="(.*)" height.*',str(user))
    users.append(user_ed)
    
# scores = []
# for score in sum(pmark_score_lists,[]):
#     score_ed = re.findall('<div class="c-rating__score">(.*)</div>',str(score))
#     scores.append(score_ed)

In [ ]:
print(len(users))
print(len(comments))
print(len(sum(pmark_score_lists,[])))

In [ ]:
import pandas as pd
df = pd.DataFrame([sum(users,[]),sum(comments,[]),sum(sum(pmark_score_lists,[]),[])],
                 index =['users','comments','scores']).T

In [ ]:
df.to_csv('Luoxiaohei.csv',encoding ='utf-8-sig')

scores_tmp = []
for i in df['scores']:
    if i != '-':
        scores_tmp.append(float(i))
<!-- #     else: -->
        scores_tmp.append(i.replace('-','0'))

### 前処理

In [ ]:
#emojiの除去
def filter_emoji(desstr, restr=''):
    """
    filter emoji
    desstr: origin str
    restr: replace str
    """
    # filter emoji
    try:
        res = re.compile(u'[\U00010000-\U0010ffff]')
    except re.error:
        res = re.compile(u'[\uD800-\uDBFF][\uDC00-\uDFFF]')
    return res.sub(restr, desstr)

In [ ]:
texts_ed = [filter_emoji(i) for i in df['comments']]

In [ ]:
#改行記号などの除去
def linebreak(desstr):
    return ''.join(desstr.replace('<br/>','').split())

In [ ]:
#顔文字の除去
def Emoticons(desstr):
    return re.sub('amp|','',desstr)
#     return re.sub('φωφ|д|ф|℃|○|〇|&amp|❓|❗','',desstr)

In [ ]:
texts_wakati = [Emoticons(i) for i in texts_wakati]

In [ ]:
texts_wakati[754]

In [ ]:
#数字の除去
def number(desstr):
    return re.sub('\d+', '', desstr)

In [ ]:
texts_wakati = [number(i) for i in texts_wakati]

In [ ]:
texts_wakati[7]

In [ ]:
#stopwords

stopwords_list = ['あそこ','あたり','あちら','あっち','あと','あな','あなた','あれ','いくつ','いつ','いま','いや','いろいろ','うち','おおまか','おまえ','おれ','がい','かく','かたち','かやの','から','がら','きた','くせ','ここ','こっち','こと','ごと','こちら','ごっちゃ','これ','これら','ごろ','さまざま','さらい','さん','しかた','しよう','すか','ずつ','すね','すべて','ぜんぶ','そう','そこ','そちら','そっち','そで','それ','それぞれ','それなり','たくさん','たち','たび','ため','だめ','ちゃ','ちゃん','てん','とおり','とき','どこ','どこか','ところ','どちら','どっか','どっち','どれ','なか','なかば','なに','など','なん','はじめ','はず','はるか','ひと','ひとつ','ふく','ぶり','べつ','へん','ぺん','ほう','ほか','まさ','まし','まとも','まま','みたい','みつ','みなさん','みんな','もと','もの','もん','やつ','よう','よそ','わけ','わたし','','ハイ','','','上','中','下','字','','','年','月','日','時','分','秒','週','火','水','木','金','土','国','都','道','府','県','市','区','町','村','','','各','第','方','何','的','度','文','者','性','体','人','他','今','部','課','係','外','類','達','気','室','口','誰','用','界','会','首','男','女','別','話','私','屋','店','家','場','等','見','際','観','段','略','例','系','論','形','間','地','員','線','点','書','品','力','法','感','作','元','手','数','彼','彼女','子','内','楽','喜','怒','哀','輪','頃','化','境','俺','奴','高','校','婦','伸','紀','誌','レ','行','列','事','士','台','集','様','所','歴','器','名','情','連','毎','式','簿','','','','','回','匹','個','席','束','歳','目','通','面','円','玉','枚','','前','後','左','右','次','先','','春','夏','秋','冬','','','','一','二','三','四','五','六','七','八','九','十','百','千','万','億','兆','','','下記','上記','時間','今回','前回','場合','一つ','年生','自分','ヶ所','ヵ所','カ所','箇所','ヶ月','ヵ月','カ月','箇月','名前','本当','確か','時点','全部','関係','近く','方法','我々','違い','多く','扱い','新た','その後','半ば','結局','様々','以前','以後','以降','未満','以上','以下','幾つ','毎日','自体','向こう','何人','手段','同じ','感じ']

def stopwords(desstr,stopwords):
    stoplist_removed_documents = []
    for i in desstr.split():
        if i not in stopwords:
            stoplist_removed_documents.append(i)
    return stoplist_removed_documents

In [ ]:
#stopwordsを除去
texts_wakati_cleaned = [stopwords(i,stopwords_list) for i in texts_wakati]

texts_wakati_cleaned_tmp = [' '.join(i) for i in texts_wakati_cleaned] 

#分かち書き列を追加
df['text_wakati'] = texts_wakati_cleaned_tmp

In [ ]:
df.head(2)

import numpy as np
print(df.isnull().any())

df_null = pd.isnull(df)
df_null = df[df == True]
print(df_null)

df = df.fillna('0')

In [ ]:
df.to_csv('羅小黒戦記.csv',encoding='utf-8-sig')

In [ ]:
df.shape

## <span style="color: red; ">感情分析</span>
    スコアと内容の乖離とのことがかるかもしれないため
    点数が低いというより、そもそも点数つけていないユーザーが多い
    
    ①極性辞書を元に感情分析を行う
    ②機械学習を元に感情分析を行う

#### ①機械学習を元に感情分析を行う
    参考＜https://blog.csdn.net/u011001084/article/details/78980299＞
    <https://www.jianshu.com/p/29aa3ad63f9d>

In [ ]:
#label付与
# def make_label(df):
#     df['sentiment'] = df['scores'].apply(lambda x:1 if float(x)>3.5 else 0)

In [ ]:
df.loc[df.scores == '-','sentiment'] = 2
df.loc[(df.scores <= '3.5')&(df.scores != '-'),'sentiment'] = 0
df.loc[df.scores > '3.5','sentiment'] = 1

In [ ]:
df['sentiment'] = df['sentiment'].replace('0',0)

In [ ]:
print(len(df[df.sentiment == 2]))
print(len(df[df.sentiment == 1]))
print(len(df[df.sentiment == 0]))

In [ ]:
df[df.sentiment == 0]

In [ ]:
df_train = df[df.sentiment != 2]
df_train 

In [ ]:
X = df_train[['text_wakati']]
Y = df_train['sentiment']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=1/3, random_state=3)

In [ ]:
X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

In [ ]:
vect = CountVectorizer()

In [ ]:
max_df = 0.8
min_df = 3

vect = CountVectorizer(max_df= max_df,
                       min_df=min_df)

In [ ]:
term_matrix = pd.DataFrame(vect.fit_transform(X_train.text_wakati).toarray(), columns=vect.get_feature_names())
term_matrix.shape

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha =1.0,class_prior = None,fit_prior=True)

In [ ]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(vect, nb)

In [ ]:
pipe.steps

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(pipe,X_train.text_wakati, Y_train, cv=5, scoring='accuracy').mean()

In [ ]:
pipe.fit(X_train.text_wakati, Y_train)

In [ ]:
Y_pred = pipe.predict(X_test.text_wakati)

In [ ]:
X_test['pred'] = Y_pred.tolist()
X_test[X_test['pred'] ==0]

In [ ]:
X_test.index.tolist()

In [ ]:
df_tmp = df[['sentiment','comments','scores']].loc[X_test.index.tolist()]
X_test['true'] = df_tmp['sentiment']
X_test['comments'] = df_tmp['comments']
X_test['scores'] = df_tmp['scores']

In [ ]:
print(len(X_test[X_test['pred'] != X_test['true']]))
X_test[X_test['pred'] != X_test['true']]

In [ ]:
from sklearn.metrics import roc_curve, auc
roc_auc = accuracy_score(Y_test, Y_pred)

In [ ]:
import matplotlib.pyplot as plt

fpr, tpr, thresholds = roc_curve(Y_test, Y_pred)

plt.plot(fpr, tpr, lw=1, label='ROC(area = %0.2f)' % (roc_auc))
plt.xlabel("FPR (False Positive Rate)")
plt.ylabel("TPR (True Positive Rate)")
plt.title("Receiver Operating Characteristic, ROC(AUC = %0.2f)"% (roc_auc))
plt.show()

In [ ]:
CM = metrics.confusion_matrix(Y_test, Y_pred)
CM

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(CM,annot=True)

In [ ]:
X_pred_michi = df.loc[df.scores == '-'][['text_wakati']]
Y_pred_michi = pipe.predict(X_pred_michi.text_wakati)

In [ ]:
Y_pred_michi 

In [ ]:
df_tmp_2 = df.loc[df.scores == '-']
df_tmp_2['pred']=Y_pred_michi.tolist()
df_tmp_2

In [ ]:
str_good= str(''.join(df[df.scores >'4'].text_wakati))
str_bad= str(''.join(df[df.scores <'3'].text_wakati))
str_normal= str(''.join(df[(df.scores <='4')&(df.scores>='3')].text_wakati))

In [ ]:
import collections
frequency_good = collections.Counter(str_good.split())
frequency_good.most_common(200)                         

In [ ]:
frequency_bad = collections.Counter(str_bad.split())
frequency_bad.most_common(200) 

In [ ]:
frequency_normal = collections.Counter(str_normal.split())
frequency_normal.most_common(200) 

In [ ]:
words_good = []
for k,v in frequency_good.most_common(200):
    words_good.append(k)

In [ ]:
from matplotlib import pyplot as plt
from wordcloud import WordCloud
font = r'C:\Users\jorey\python\UDDigiKyokashoN-R.ttc'
stopwords =set(['映画','アニメ','シャオヘイ','し','いう','い','ある','いる','ない','さ','いい','する'])

wc = WordCloud(background_color = 'black',
               font_path =font,
               width = 1000,
               height = 800,
               stopwords=stopwords
               ).generate(str(words_good))
plt.imshow(wc)
wc.to_file('good.jpg')

In [ ]:
#(1)すべての単語を特徴にする
def bag_of_words(words):  
    return dict([(word, True) for word in words]) 

In [ ]:
df[(df.scores <= '3.5') & (df.scores !='-')]

In [ ]:
df[df.scores > '3.5']

In [ ]:
feature_words_good =bag_of_words(' '.join(df[df.scores > '3.5'].text_wakati.values).split())
feature_words_bad =bag_of_words(' '.join(df[(df.scores <= '3.5') & (df.scores !='-')].text_wakati.values).split())

In [ ]:
print("👍の評価:" + str(len(feature_words_good)))

print("========================================================================")

print("👎の評価:" + str(len(feature_words_bad)))

In [ ]:
feature_words_good

In [ ]:
#(2)2つの単語のペア（bigrams)を特徴にする
import nltk  
from nltk.collocations import BigramCollocationFinder  
from nltk.metrics import BigramAssocMeasures  

In [ ]:
def bigram(words, score_fn=BigramAssocMeasures.chi_sq, n=1000):  
    bigram_finder = BigramCollocationFinder.from_words(words) #単語ペアの作成  
    bigrams = bigram_finder.nbest(score_fn, n) #カイ二乗、スコア上位1000の単語ペアをセレクト 
    newBigrams = [u+ ' ' + v for (u,v) in bigrams]
    return bag_of_words(newBigrams)

In [ ]:
biggram_words_good = bigram(feature_words_good)
biggram_words_bad = bigram(feature_words_bad)

In [ ]:
print("👍の評価:" + str(biggram_words_good))

print("========================================================================")

print("👎の評価:" + str(biggram_words_bad))

In [ ]:
def bigram_words(words,score_fn=BigramAssocMeasures.chi_sq,n=1000):
    bigram_finder=BigramCollocationFinder.from_words(words)
    bigrams = bigram_finder.nbest(score_fn,n)
    newBigrams = [u + ' ' + v for (u,v) in bigrams]
    a = bag_of_words(words)
    b = bag_of_words(newBigrams)
    a.update(b) 
    return a 

In [ ]:
biggram_good = bigram_words(feature_words_good)
biggram_bad = bigram_words(feature_words_bad)

In [ ]:
print("👍の評価:" + str(biggram_good))

print("========================================================================")

print("👎の評価:" + str(biggram_bad))

In [ ]:
#(3)コーパス内単語ごとの情報量を計算する
from nltk.probability import FreqDist, ConditionalFreqDist  
import itertools
def create_word_scores():
    #辞書の形をリストに整理する
    poswords = list(itertools.chain(feature_words_good))
    negwords = list(itertools.chain(feature_words_bad))
    
    #単語の頻度を統計する
    word_fd = FreqDist()
    cond_word_fd = ConditionalFreqDist()
    for word in poswords:
        word_fd[word] += 1  
        cond_word_fd['pos'][word] += 1
    for word in negwords:
        word_fd[word] += 1
        cond_word_fd['neg'][word] +=1  
    
    #ポジとネガ単語数のカウント
    pos_word_count = cond_word_fd['pos'].N()
    neg_word_count = cond_word_fd['neg'].N() 
    total_word_count = pos_word_count + neg_word_count  
    
    word_scores = {}  
    for word, freq in word_fd.items():  
        pos_score = BigramAssocMeasures.chi_sq(cond_word_fd['pos'][word], (freq, pos_word_count), total_word_count) #计算积极词的卡方统计量，这里也可以计算互信息等其它统计量  
        neg_score = BigramAssocMeasures.chi_sq(cond_word_fd['neg'][word], (freq, neg_word_count), total_word_count) #同理  
        word_scores[word] = pos_score + neg_score #一个词的信息量等于积极卡方统计量加上消极卡方统计量  
    
    return word_scores

In [ ]:
def create_word_bigram_scores(): 
    posWords = list(itertools.chain(biggram_words_good))  
    negWords = list(itertools.chain(biggram_words_bad))  

    bigram_finder = BigramCollocationFinder.from_words(posWords)  
    bigram_finder = BigramCollocationFinder.from_words(negWords)  
    posBigrams = bigram_finder.nbest(BigramAssocMeasures.chi_sq, 5000)  
    negBigrams = bigram_finder.nbest(BigramAssocMeasures.chi_sq, 5000)  

    pos = posWords + posBigrams  
    neg = negWords + negBigrams

    word_fd = FreqDist()  
    cond_word_fd = ConditionalFreqDist()  
    for word in pos:  
        word_fd[word] += 1  
        cond_word_fd['pos'][word] += 1 
    for word in neg:  
        word_fd[word] += 1
        cond_word_fd['neg'][word] += 1  

    pos_word_count = cond_word_fd['pos'].N()  
    neg_word_count = cond_word_fd['neg'].N()  
    total_word_count = pos_word_count + neg_word_count  

    word_scores = {}  
    for word, freq in word_fd.items():  
        pos_score = BigramAssocMeasures.chi_sq(cond_word_fd['pos'][word], (freq, pos_word_count), total_word_count)  
        neg_score = BigramAssocMeasures.chi_sq(cond_word_fd['neg'][word], (freq, neg_word_count), total_word_count)  
        word_scores[word] = pos_score + neg_score  

    return word_scores 

In [ ]:
create_word_bigram_scores()

In [ ]:
pd.DataFrame(words_score,index = ['words','socre']).T.to_csv('words_score_xiaohei.csv',encoding='utf-8-sig')

In [ ]:
def find_best_words(word_scores, number):
    best_vals = sorted(word_scores.items(),key = lambda w_s:w_s[1],reverse = True)[:number]
#     best_vals = sorted(word_scores.iteritems(), key=lambda (w, s):(w,s), reverse=True)[:number]
#     best_vals = sorted(word_scores.items(), key=lambda item:item[1],  reverse=True)[:number]
    best_words = set([w for w, s in best_vals])
    return best_words

In [ ]:
word_scores_1 = create_word_scores()  
word_scores_2 = create_word_bigram_scores() 

In [ ]:
best_words = find_best_words(word_scores_1,100)

In [ ]:
def best_word_features(words):  
    #load_data()
    #word_scores = create_word_bigram_scores()
    global best_words
    #best_words = find_best_words(word_scores,7500)
    return dict([(word, True) for word in words if word in best_words])  

In [ ]:
best_word_scores_1=best_word_features(word_scores_1)
best_word_scores_1

In [ ]:
best_words = find_best_words(word_scores_2,100)
best_word_scores_2=best_word_features(word_scores_2)
best_word_scores_2

In [ ]:
def pos_feature(feature_extraction_method):
    posFeatures =[]
    for i in pos_review:
        posWords = [feature_extraction_method(i),'pos']
        posFeatures.append(posWords)
    return posFeatures

# 廃止

In [ ]:
pmark_dict = dict(zip(sum(users,[]),sum(comments,[])))

In [ ]:
import pandas as pd
df_tmp = pd.DataFrame(pmark_dict,index=['comments']).T

df = df_tmp.reset_index().rename(columns={'index':'user'})

In [ ]:
print(len(scores))
print(len(df))

In [ ]:
df['comments'][0] 
#絵文字
#改行など
#感情溢れすぎ表現

texts_ed[0]